In [1]:
import sys
import pandas as pd
import numpy as np
import math
from scipy.special import softmax
from scipy.optimize import minimize, LinearConstraint
from scipy.spatial.distance import braycurtis
import multiprocessing
from concurrent.futures import ProcessPoolExecutor, as_completed
from imports import *
from importlib import reload
import matplotlib.pyplot as plt
import seaborn as sns
# # adding Folder_2 to the system path
# sys.path.insert(0, r'C:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\code')
# # sys.path.insert(0, r'C:\Users\yuvald\Documents\Uni\סמסטר ב\workshop_microbiome\code')

from imports import  *

import social_model_class_v2
reload(social_model_class_v2)

import time


In [9]:
data_path = r"train_data.csv"
metadata_path = r"train_metadata.csv"



[1.0,
 0.1,
 0.010000000000000002,
 0.0010000000000000002,
 0.00010000000000000002,
 1.0000000000000003e-05,
 1.0000000000000004e-06,
 1.0000000000000004e-07,
 1.0000000000000005e-08,
 1.0000000000000005e-09,
 1.0000000000000006e-10,
 1.0000000000000006e-11,
 1.0000000000000006e-12,
 1.0000000000000007e-13,
 1.0000000000000008e-14,
 1.0000000000000009e-15,
 1.0000000000000008e-16,
 1.000000000000001e-17,
 1.000000000000001e-18,
 1.000000000000001e-19,
 1.0000000000000011e-20,
 1.0000000000000012e-21,
 1.0000000000000012e-22,
 1.0000000000000013e-23,
 1.0000000000000014e-24,
 1.0000000000000014e-25,
 1.0000000000000015e-26,
 1.0000000000000015e-27,
 1.0000000000000015e-28,
 1.0000000000000016e-29,
 1.0000000000000017e-30,
 1.0000000000000016e-31,
 1.0000000000000018e-32,
 1.0000000000000018e-33,
 1.0000000000000019e-34,
 1.0000000000000019e-35,
 1.000000000000002e-36,
 1.0000000000000022e-37,
 1.000000000000002e-38,
 1.0000000000000022e-39,
 1.0000000000000022e-40,
 1.0000000000000023e-

$$\Delta_t  \space windowing$$

In [4]:
model = social_model_class_v2.superModel(data_path, metadata_path)

assert np.all(model.metadata_df.index.isin(model.data_df.index))
t = time.time()
model.baboons["Baboon_101"].fit(np.zeros(61), 0)
print(time.time() -t)
model.baboons["Baboon_101"].beta_

c:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\Milestone2\social_model_class_v2.py:295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata_clean['collection_date'] = pd.to_datetime(metadata_clean['collection_date'])
Traceback (most recent call last):
  File "c:\Users\tomer\AppData\Local\Programs\Python\Python311\Lib\multiprocessing\queues.py", line 246, in _feed
    send_bytes(obj)
  File "c:\Users\tomer\AppData\Local\Programs\Python\Python311\Lib\multiprocessing\connection.py", line 184, in send_bytes
    self._check_closed()
  File "c:\Users\tomer\AppData\Local\Programs\Python\Python311\Lib\multiprocessing\connection.py", line 137, in _check_closed
    raise OSError("handle is closed")
OSError: handle is closed
Traceback (most r

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [16]:
# model = social_model_class_v2.superModel(data_path, metadata_path)
# model.alpha_ = np.array([0]*61)
# for baboon in model.baboons.values():
#     baboon.fit(0.11, np.zeros(61))

# mat = np.array([baboon.beta_ for baboon in model.baboons.values()])

# sns.heatmap(pd.DataFrame(mat,columns=baboon.data_I.columns, index = model.baboons.keys()), cmap='viridis')
# plt.show()

## cross validation (4 - fold)

In [17]:

baboons = model.baboons
dict_list = list(model.baboons.items())
np.random.shuffle(dict_list)
groups = [dict(group) for group in np.array_split(dict_list,4)]

data_df = pd.read_csv(data_path, index_col=0)
metadata_df = pd.read_csv(metadata_path, index_col=0)

for i in range(4):
    train_metatdata = metadata_df[~metadata_df["baboon_id"].isin(groups[i].keys())]
    train_data = data_df.loc[train_metatdata.index]
    known_metadata = metadata_df[metadata_df['baboon_id'].isin(groups[i].keys())]
    indicies = []
    for baboon in groups[i].values():
        indicies.extend(baboon.metadata_I.index[:10])

    known_data = data_df.loc[indicies]
    true_data = data_df.loc[known_metadata[~np.isin(known_metadata.index, indicies)].index]


    train_data.to_csv(fr".\cross_val_data\train_data_{i}.csv")
    train_metatdata.to_csv(fr".\cross_val_data\train_metadata_{i}.csv")
    known_data.to_csv(fr".\cross_val_data\known_data_{i}.csv")
    known_metadata.to_csv(fr".\cross_val_data\known_metadata_{i}.csv")
    true_data.to_csv(fr".\cross_val_data\true_data_{i}.csv")
    

In [18]:

baboons = model.baboons
dict_list = list(model.baboons.items())
np.random.shuffle(dict_list)
groups = [dict(group) for group in np.array_split(dict_list,4)]
metadata_df, data_df = social_model_class_v2.preprocessing(data_path, metadata_path)

for i in range(4):
    train_metatdata = metadata_df[~metadata_df["baboon_id"].isin(groups[i].keys())]
    train_data = data_df.loc[train_metatdata.index]
    known_metadata = metadata_df[metadata_df['baboon_id'].isin(groups[i].keys())]
    indicies = []
    for baboon in groups[i].values():
        indicies.extend(baboon.metadata_I.index[:10])

    known_data = data_df.loc[indicies]
    true_data = data_df.loc[known_metadata[~np.isin(known_metadata.index, indicies)].index]


    train_data.to_csv(fr".\cross_val_data\train_data_{i}.csv")
    train_metatdata.to_csv(fr".\cross_val_data\train_metadata_{i}.csv")
    known_data.to_csv(fr".\cross_val_data\known_data_{i}.csv")
    known_metadata.to_csv(fr".\cross_val_data\known_metadata_{i}.csv")
    true_data.to_csv(fr".\cross_val_data\true_data_{i}.csv")
    
for i in range(4):
    model = social_model_class_v2.superModel(fr".\cross_val_data\train_data_{i}.csv", fr".\cross_val_data\train_metadata_{i}.csv")
    model.alpha_ = np.array([0]*61)
    
    cpus = max(1, min(multiprocessing.cpu_count() - 2, len(model.baboons)))
    
       # with ProcessPoolExecutor(cpus) as executor:
    #     futures = [executor.submit(baboon.fit, model.alpha_) for baboon in model.baboons.values()]
    #     for future in as_completed(futures):
    #         pass
    for baboon in model.baboons.values():
        baboon.beta= np.random.rand(61)
    print(f"predicting {i}")

    model.add_new_data(fr".\cross_val_data\known_data_{i}.csv", fr".\cross_val_data\known_metadata_{i}.csv")

    print(f"new data added {i}")
    predictions_noniterative = model.predict(groups[i].keys(), iterative=False)
    predictions_iterative = model.predict(groups[i].keys(), iterative=True)
    predictions_noniterative.to_csv(fr".\cross_val_data\predictions_noniterative_{i}.csv")
    predictions_iterative.to_csv(fr".\cross_val_data\predictions_iterative_{i}.csv")

c:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\Milestone2\social_model_class_v2.py:288: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata_clean['collection_date'] = pd.to_datetime(metadata_clean['collection_date'])


predicting 0
new data added 0


KeyError: "None of [Index(['sample_11411-TCGAAGACGTAT-396', 'sample_12051-AGTAGTTTCCTT-408',\n       'sample_12051-TGGCGCACGGAC-408', 'sample_12049-CTAGAGCTCCCA-407',\n       'sample_11413-CCTAGCCCTAGC-397', 'sample_11413-CCTGTGTTGGTG-397',\n       'sample_12050-ACCGTCTTTCTC-407', 'sample_11406-GCGCGAATGGTG-394',\n       'sample_12052-GTCACATCACGA-408', 'sample_11406-GCAGGGTCGAAC-394',\n       'sample_12052-TCCGAGTCACCA-408', 'sample_12052-TTAAACCGCGCC-408',\n       'sample_12052-TTGTAGCCGACA-408', 'sample_11413-CACTAGACCCAC-397',\n       'sample_11411-GGCGCCATCTCT-396', 'sample_11407-TCGGTTACGCTG-394',\n       'sample_12053-GATGCCTAATGA-409', 'sample_12052-TACAGCGCATAC-408',\n       'sample_11408-TCACTGCTAGGA-395', 'sample_11408-TATGCTCTCTCA-395',\n       'sample_11412-GTTCGGTGTCCA-397', 'sample_12050-TACATATCTACA-407',\n       'sample_11407-TGCCAGACCACT-394', 'sample_11408-AGTCATCGAATG-395',\n       'sample_12052-TATGCTCTCTCA-408', 'sample_11407-TCAGACCAACTG-394',\n       'sample_120

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(10, 15))

for i in range(4):
    # Load data
    predictions_noniterative = pd.read_csv(fr".\cross_val_data\predictions_noniterative_{i}.csv", index_col=0)
    predictions_iterative = pd.read_csv(fr".\cross_val_data\predictions_iterative_{i}.csv", index_col=0)
    true_data = pd.read_csv(fr".\cross_val_data\true_data_{i}.csv", index_col=0)

    # Initialize lists to hold Bray-Curtis distances for each group
    bc_noniterative = []
    bc_iterative = []

    # Loop through each baboon
    for baboon in groups[i].keys():
        score_noniter =[] 
        score_iter =[]
        for idx in np.intersect1d(true_data.index, groups[i][baboon].metadata_I.index):
            
            # Calculate Bray-Curtis scores
            score_noniter.append(braycurtis(true_data.loc[idx], predictions_noniterative.loc[idx]))
            score_iter.append(braycurtis(true_data.loc[idx], predictions_iterative.loc[idx]))

            # Append the scores to respective lists
        bc_noniterative.append(np.mean(score_noniter))
        bc_iterative.append(np.mean(score_iter))

    # Plot KDE plots for both noniterative and iterative scores
    sns.kdeplot(bc_noniterative, ax=axes[i, 0], label="noniterative")
    sns.kdeplot(bc_iterative, ax=axes[i, 1], label="iterative")
    axes[i, 0].set_xlim(0, 1)
    axes[i, 1].set_xlim(0, 1)
    axes[i, 0].vlines(np.mean(bc_noniterative), 0, 10, color='red', linestyle='--')
    axes[i, 1].vlines(np.mean(bc_iterative), 0, 10, color='red', linestyle='--')

    # Set titles
    axes[i, 0].set_title(f"Non-iterative (Group {i+1}), mean={np.mean(bc_noniterative):.3f}")
    axes[i, 1].set_title(f"Iterative (Group {i+1}), mean={np.mean(bc_iterative):.3f}")

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 7))
        
for i in range(4):
    # Load data
    predictions_noniterative = pd.read_csv(fr".\cross_val_data\predictions_noniterative_{i}.csv", index_col=0)
    predictions_iterative = pd.read_csv(fr".\cross_val_data\predictions_iterative_{i}.csv", index_col=0)
    true_data = pd.read_csv(fr".\cross_val_data\true_data_{i}.csv", index_col=0)

    # Initialize lists to hold Bray-Curtis distances for each group
    bc_noniterative = []
    bc_iterative = []

    # Loop through each baboon
    for baboon in groups[i].keys():
        score_noniter =[] 
        score_iter =[]
        for idx in np.intersect1d(true_data.index, groups[i][baboon].metadata_I.index):
            
            # Calculate Bray-Curtis scores
            score_noniter.append(braycurtis(true_data.loc[idx], predictions_noniterative.loc[idx]))
            score_iter.append(braycurtis(true_data.loc[idx], predictions_iterative.loc[idx]))

            # Append the scores to respective lists
        bc_noniterative.append(np.mean(score_noniter))
        bc_iterative.append(np.mean(score_iter))


    # Plot KDE for non-iterative and iterative results across group
    sns.kdeplot(bc_noniterative, ax=axes[0], label=f"Group {i}, mean={np.mean(bc_noniterative):.3f}")
    sns.kdeplot(bc_iterative, ax=axes[1],  label=f"Group {i}, mean={np.mean(bc_iterative):.3f}")

# Set titles and labels
axes[0].set_title("Non-iterative")
axes[1].set_title("Iterative")
axes[0].set_xlabel("Bray-Curtis Distance")
axes[1].set_xlabel("Bray-Curtis Distance")
axes[0].legend()
axes[1].legend()

plt.tight_layout()
plt.show()

### Only first 2

In [48]:

baboons = model.baboons
dict_list = list(model.baboons.items())
np.random.shuffle(dict_list)
groups = [dict(group) for group in np.array_split(dict_list,4)]

data_df = pd.read_csv(data_path, index_col=0)
metadata_df = pd.read_csv(metadata_path, index_col=0)

for i in range(4):
    train_metatdata = metadata_df[~metadata_df["baboon_id"].isin(groups[i].keys())]
    train_data = data_df.loc[train_metatdata.index]
    known_metadata = metadata_df[metadata_df['baboon_id'].isin(groups[i].keys())]
    indicies = []
    for baboon in groups[i].values():
        indicies.extend(baboon.metadata_I.index[:2])

    known_data = data_df.loc[indicies]
    true_data = data_df.loc[known_metadata[~np.isin(known_metadata.index, indicies)].index]


    train_data.to_csv(fr".\cross_val_data\train_data_{i}.csv")
    train_metatdata.to_csv(fr".\cross_val_data\train_metadata_{i}.csv")
    known_data.to_csv(fr".\cross_val_data\known_data_{i}.csv")
    known_metadata.to_csv(fr".\cross_val_data\known_metadata_{i}.csv")
    true_data.to_csv(fr".\cross_val_data\true_data_{i}.csv")
    

In [ ]:

baboons = model.baboons
dict_list = list(model.baboons.items())
np.random.shuffle(dict_list)
groups = [dict(group) for group in np.array_split(dict_list,4)]
metadata_df, data_df = social_model_class_v2.preprocessing(data_path, metadata_path)

for i in range(4):
    train_metatdata = metadata_df[~metadata_df["baboon_id"].isin(groups[i].keys())]
    train_data = data_df.loc[train_metatdata.index]
    known_metadata = metadata_df[metadata_df['baboon_id'].isin(groups[i].keys())]
    indicies = []
    for baboon in groups[i].values():
        indicies.extend(baboon.metadata_I.index[:2])

    known_data = data_df.loc[indicies]
    true_data = data_df.loc[known_metadata[~np.isin(known_metadata.index, indicies)].index]


    train_data.to_csv(fr".\cross_val_data\train_data_{i}.csv")
    train_metatdata.to_csv(fr".\cross_val_data\train_metadata_{i}.csv")
    known_data.to_csv(fr".\cross_val_data\known_data_{i}.csv")
    known_metadata.to_csv(fr".\cross_val_data\known_metadata_{i}.csv")
    true_data.to_csv(fr".\cross_val_data\true_data_{i}.csv")
    
for i in range(4):
    model = social_model_class_v2.superModel(fr".\cross_val_data\train_data_{i}.csv", fr".\cross_val_data\train_metadata_{i}.csv")
    model.alpha_ = np.array([0]*61)
    
    cpus = max(1, min(multiprocessing.cpu_count() - 2, len(model.baboons)))
    with ProcessPoolExecutor(cpus) as executor:
        futures = [executor.submit(baboon.fit, model.alpha_) for baboon in model.baboons.values()]
        for future in as_completed(futures):
            pass
    
    print(f"predicting {i}")
    
    model.add_new_data(fr".\cross_val_data\known_data_{i}.csv", fr".\cross_val_data\known_metadata_{i}.csv")

    print(f"new data added {i}")
    predictions_noniterative = model.predict(groups[i].keys(), iterative=False)
    predictions_iterative = model.predict(groups[i].keys(), iterative=True)
    predictions_noniterative.to_csv(fr".\cross_val_data\predictions_noniterative_{i}.csv")
    predictions_iterative.to_csv(fr".\cross_val_data\predictions_iterative_{i}.csv")

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(10, 15))

for i in range(4):
    # Load data
    predictions_noniterative = pd.read_csv(fr".\cross_val_data\predictions_noniterative_{i}.csv", index_col=0)
    predictions_iterative = pd.read_csv(fr".\cross_val_data\predictions_iterative_{i}.csv", index_col=0)
    true_data = pd.read_csv(fr".\cross_val_data\true_data_{i}.csv", index_col=0)

    # Initialize lists to hold Bray-Curtis distances for each group
    bc_noniterative = []
    bc_iterative = []

    # Loop through each baboon
    for baboon in groups[i].keys():
        score_noniter =[] 
        score_iter =[]
        for idx in np.intersect1d(true_data.index, groups[i][baboon].metadata_I.index):
            
            # Calculate Bray-Curtis scores
            score_noniter.append(braycurtis(true_data.loc[idx], predictions_noniterative.loc[idx]))
            score_iter.append(braycurtis(true_data.loc[idx], predictions_iterative.loc[idx]))

            # Append the scores to respective lists
        bc_noniterative.append(np.mean(score_noniter))
        bc_iterative.append(np.mean(score_iter))


    # Plot KDE for non-iterative and iterative results across group
    sns.kdeplot(bc_noniterative, ax=axes[i, 0], label="noniterative")
    sns.kdeplot(bc_iterative, ax=axes[i, 1], label="iterative")
    axes[i, 0].set_xlim(0, 1)
    axes[i, 1].set_xlim(0, 1)
    axes[i, 0].vlines(np.mean(bc_noniterative), 0, 10, color='red', linestyle='--')
    axes[i, 1].vlines(np.mean(bc_iterative), 0, 10, color='red', linestyle='--')

    # Set titles
    axes[i, 0].set_title(f"Non-iterative (Group {i+1}) Super Short (first 2), mean={np.mean(bc_noniterative):.3f}")
    axes[i, 1].set_title(f"Iterative (Group {i+1}) Super Short (first 2), mean={np.mean(bc_iterative):.3f}")

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 7))

# Initialize lists to accumulate Bray-Curtis distances for all groups
bc_noniterative = []
bc_iterative = []

for i in range(4):
    # Load data
    predictions_noniterative = pd.read_csv(fr".\cross_val_data\predictions_noniterative_{i}.csv", index_col=0)
    predictions_iterative = pd.read_csv(fr".\cross_val_data\predictions_iterative_{i}.csv", index_col=0)
    true_data = pd.read_csv(fr".\cross_val_data\true_data_{i}.csv", index_col=0)

    # Initialize lists to hold Bray-Curtis distances for each group
    bc_noniterative = []
    bc_iterative = []

    # Loop through each baboon
    for baboon in groups[i].keys():
        score_noniter =[] 
        score_iter =[]
        for idx in np.intersect1d(true_data.index, groups[i][baboon].metadata_I.index):
            
            # Calculate Bray-Curtis scores
            score_noniter.append(braycurtis(true_data.loc[idx], predictions_noniterative.loc[idx]))
            score_iter.append(braycurtis(true_data.loc[idx], predictions_iterative.loc[idx]))

            # Append the scores to respective lists
        bc_noniterative.append(np.mean(score_noniter))
        bc_iterative.append(np.mean(score_iter))


    # Plot KDE for non-iterative and iterative results across group
    sns.kdeplot(bc_noniterative, ax=axes[0], label=f"Group {i}, mean={np.mean(bc_noniterative):.3f}")
    sns.kdeplot(bc_iterative, ax=axes[1],  label=f"Group {i}, mean={np.mean(bc_iterative):.3f}")

# Set titles and labels
axes[0].set_title("Non-iterative Super Short (first 2)")
axes[1].set_title("Iterative Super Short (first 2)")
axes[0].set_xlabel("Bray-Curtis Distance")
axes[1].set_xlabel("Bray-Curtis Distance")
axes[0].legend()
axes[1].legend()

plt.tight_layout()
plt.show()

### all but last 10

In [52]:

baboons = model.baboons
dict_list = list(model.baboons.items())
np.random.shuffle(dict_list)
groups = [dict(group) for group in np.array_split(dict_list,4)]

data_df = pd.read_csv(data_path, index_col=0)
metadata_df = pd.read_csv(metadata_path, index_col=0)

for i in range(4):
    train_metatdata = metadata_df[~metadata_df["baboon_id"].isin(groups[i].keys())]
    train_data = data_df.loc[train_metatdata.index]
    known_metadata = metadata_df[metadata_df['baboon_id'].isin(groups[i].keys())]
    indicies = []
    for baboon in groups[i].values():
        indicies.extend(baboon.metadata_I.index[:-10])

    known_data = data_df.loc[indicies]
    true_data = data_df.loc[known_metadata[~np.isin(known_metadata.index, indicies)].index]


    train_data.to_csv(fr".\cross_val_data\train_data_{i}.csv")
    train_metatdata.to_csv(fr".\cross_val_data\train_metadata_{i}.csv")
    known_data.to_csv(fr".\cross_val_data\known_data_{i}.csv")
    known_metadata.to_csv(fr".\cross_val_data\known_metadata_{i}.csv")
    true_data.to_csv(fr".\cross_val_data\true_data_{i}.csv")
    

In [ ]:

baboons = model.baboons
dict_list = list(model.baboons.items())
np.random.shuffle(dict_list)
groups = [dict(group) for group in np.array_split(dict_list,4)]
metadata_df, data_df = social_model_class_v2.preprocessing(data_path, metadata_path)

for i in range(4):
    train_metatdata = metadata_df[~metadata_df["baboon_id"].isin(groups[i].keys())]
    train_data = data_df.loc[train_metatdata.index]
    known_metadata = metadata_df[metadata_df['baboon_id'].isin(groups[i].keys())]
    indicies = []
    for baboon in groups[i].values():
        indicies.extend(baboon.metadata_I.index[:10])

    known_data = data_df.loc[indicies]
    true_data = data_df.loc[known_metadata[~np.isin(known_metadata.index, indicies)].index]


    train_data.to_csv(fr".\cross_val_data\train_data_{i}.csv")
    train_metatdata.to_csv(fr".\cross_val_data\train_metadata_{i}.csv")
    known_data.to_csv(fr".\cross_val_data\known_data_{i}.csv")
    known_metadata.to_csv(fr".\cross_val_data\known_metadata_{i}.csv")
    true_data.to_csv(fr".\cross_val_data\true_data_{i}.csv")
    
for i in range(4):
    model = social_model_class_v2.superModel(fr".\cross_val_data\train_data_{i}.csv", fr".\cross_val_data\train_metadata_{i}.csv")
    model.alpha_ = np.array([0]*61)
    
    cpus = max(1, min(multiprocessing.cpu_count() - 2, len(model.baboons)))
    with ProcessPoolExecutor(cpus) as executor:
        futures = [executor.submit(baboon.fit, model.alpha_) for baboon in model.baboons.values()]
        for future in as_completed(futures):
            pass
    
    print(f"predicting {i}")
    
    model.add_new_data(fr".\cross_val_data\known_data_{i}.csv", fr".\cross_val_data\known_metadata_{i}.csv")

    print(f"new data added {i}")
    predictions_noniterative = model.predict(groups[i].keys(), iterative=False)
    predictions_iterative = model.predict(groups[i].keys(), iterative=True)
    predictions_noniterative.to_csv(fr".\cross_val_data\predictions_noniterative_{i}.csv")
    predictions_iterative.to_csv(fr".\cross_val_data\predictions_iterative_{i}.csv")

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(10, 15))

for i in range(4):
    # Load data
    predictions_noniterative = pd.read_csv(fr".\cross_val_data\predictions_noniterative_{i}.csv", index_col=0)
    predictions_iterative = pd.read_csv(fr".\cross_val_data\predictions_iterative_{i}.csv", index_col=0)
    true_data = pd.read_csv(fr".\cross_val_data\true_data_{i}.csv", index_col=0)

    # Initialize lists to hold Bray-Curtis distances for each group
    bc_noniterative = []
    bc_iterative = []

    # Loop through each baboon
    for baboon in groups[i].keys():
        score_noniter =[] 
        score_iter =[]
        for idx in np.intersect1d(true_data.index, groups[i][baboon].metadata_I.index):
            
            # Calculate Bray-Curtis scores
            score_noniter.append(braycurtis(true_data.loc[idx], predictions_noniterative.loc[idx]))
            score_iter.append(braycurtis(true_data.loc[idx], predictions_iterative.loc[idx]))

            # Append the scores to respective lists
        bc_noniterative.append(np.mean(score_noniter))
        bc_iterative.append(np.mean(score_iter))


    # Plot KDE for non-iterative and iterative results across group
    sns.kdeplot(bc_noniterative, ax=axes[i, 0], label="noniterative")
    sns.kdeplot(bc_iterative, ax=axes[i, 1], label="iterative")
    axes[i, 0].set_xlim(0, 1)
    axes[i, 1].set_xlim(0, 1)
    axes[i, 0].vlines(np.mean(bc_noniterative), 0, 3.5, color='red', linestyle='--')
    axes[i, 1].vlines(np.mean(bc_iterative), 0, 3.5, color='red', linestyle='--')

    # Set titles
    axes[i, 0].set_title(f"Non-iterative (Group {i+1}) Long (all but last 10), mean={np.mean(bc_noniterative):.3f}")
    axes[i, 1].set_title(f"Iterative (Group {i+1}) Long (all but last 10), mean={np.mean(bc_iterative):.3f}")

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 7))

for i in range(4):
    # Load data
    predictions_noniterative = pd.read_csv(fr".\cross_val_data\predictions_noniterative_{i}.csv", index_col=0)
    predictions_iterative = pd.read_csv(fr".\cross_val_data\predictions_iterative_{i}.csv", index_col=0)
    true_data = pd.read_csv(fr".\cross_val_data\true_data_{i}.csv", index_col=0)

    # Initialize lists to hold Bray-Curtis distances for each group
    bc_noniterative = []
    bc_iterative = []

    # Loop through each baboon
    for baboon in groups[i].keys():
        score_noniter =[] 
        score_iter =[]
        for idx in np.intersect1d(true_data.index, groups[i][baboon].metadata_I.index):
            
            # Calculate Bray-Curtis scores
            score_noniter.append(braycurtis(true_data.loc[idx], predictions_noniterative.loc[idx]))
            score_iter.append(braycurtis(true_data.loc[idx], predictions_iterative.loc[idx]))

            # Append the scores to respective lists
        bc_noniterative.append(np.mean(score_noniter))
        bc_iterative.append(np.mean(score_iter))


    # Plot KDE for non-iterative and iterative results across group
    sns.kdeplot(bc_noniterative_all, ax=axes[0], label=f"Group {i}, mean={np.mean(bc_noniterative):.3f}")
    sns.kdeplot(bc_iterative_all, ax=axes[1],  label=f"Group {i}, mean={np.mean(bc_iterative):.3f}")

# Set titles and labels
axes[0].set_title("Non-iterative Long (all but last 10)")
axes[1].set_title("Iterative Long (all but last 10)")
axes[0].set_xlabel("Bray-Curtis Distance")
axes[1].set_xlabel("Bray-Curtis Distance")
axes[0].legend()
axes[1].legend()

plt.tight_layout()
plt.show()